In [1]:
import json
import os
import pandas as pd
import ray

from pprint import pprint 

from env import QdTreeEnv
from qdtree import Workload
from qdtree.schema import ensure_data_schema
from policy import QdTreePolicy

from ray.rllib.env.env_context import EnvContext
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray import air, tune


Parameters

In [2]:
# We use sf = 1 for TPC-H. The paper uses sf = 1000
BASE_PATH = "../data/tpc-h/"
# The sampling rate in the paper is 0.1
SAMPLING_RATE = 0.01
# The min block size in the paper is 100K, we use 100 to scale down according to the sf
MIN_BLOCK_SIZE = 100 * SAMPLING_RATE

# Load the workload

In [3]:
WORKLOAD_PATH = os.path.join(BASE_PATH, "workload.json")
with open(WORKLOAD_PATH, "r") as f:
    workload = Workload(json.load(f))

workload._queries

{'1.1.sql': l_shipdate <= 903247200,
 '1.5.sql': l_shipdate <= 905666400,
 '1.3.sql': l_shipdate <= 907048800,
 '1.2.sql': l_shipdate <= 902556000,
 '1.4.sql': l_shipdate <= 906357600,
 '3.1.sql':  and(o_orderdate < 796287600, l_shipdate > 796287600),
 '3.4.sql':  and(o_orderdate < 795250800, l_shipdate > 795250800),
 '3.2.sql':  and(o_orderdate < 795078000, l_shipdate > 795078000),
 '3.3.sql':  and(o_orderdate < 796460400, l_shipdate > 796460400),
 '3.5.sql':  and(o_orderdate < 796633200, l_shipdate > 796633200),
 '4.3.sql':  and(o_orderdate >= 865144800, o_orderdate < 873093600),
 '4.2.sql':  and(o_orderdate >= 783673200, o_orderdate < 791622000),
 '4.1.sql':  and(o_orderdate >= 854780400, o_orderdate < 862466400),
 '4.5.sql':  and(o_orderdate >= 875685600, o_orderdate < 883638000),
 '4.4.sql':  and(o_orderdate >= 794041200, o_orderdate < 801986400),
 '5.4.sql':  and(o_orderdate >= 725871600, o_orderdate < 757407600),
 '5.2.sql':  and(o_orderdate >= 725871600, o_orderdate < 757407600

# Load and sample data

In [4]:
DATA_PATH = os.path.join(BASE_PATH, "sf1/denormalized.parquet")
all_data = pd.read_parquet(DATA_PATH).sample(frac=0.1)
all_data.columns = all_data.columns.str.lower()
all_data = ensure_data_schema(all_data, workload.schema)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600122 entries, 269621 to 5769231
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   l_orderkey        600122 non-null  int64  
 1   l_partkey         600122 non-null  int64  
 2   l_suppkey         600122 non-null  int64  
 3   l_linenumber      600122 non-null  int64  
 4   l_quantity        600122 non-null  float64
 5   l_extendedprice   600122 non-null  float64
 6   l_discount        600122 non-null  float64
 7   l_tax             600122 non-null  float64
 8   l_shipdate        600122 non-null  int64  
 9   l_commitdate      600122 non-null  int64  
 10  l_receiptdate     600122 non-null  int64  
 11  o_orderkey        600122 non-null  int64  
 12  o_custkey         600122 non-null  int64  
 13  o_totalprice      600122 non-null  float64
 14  o_orderdate       600122 non-null  int64  
 15  o_shippriority    600122 non-null  int64  
 16  c_custkey     

In [5]:
data = all_data.sample(frac=SAMPLING_RATE)
len(data)

6001

# Set up the environment

In [6]:
ray.init(local_mode=False, ignore_reinit_error=True,num_cpus=8)

2024-05-19 19:24:24,948	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.3.1


In [7]:
env_config = {
    "workload": workload,
    "data": data,
    "min_leaf_size": MIN_BLOCK_SIZE,
}

config = (
    PPOConfig()
    .environment(
        QdTreeEnv,
        env_config=env_config,
    )
    .framework("torch")
    .rollouts(num_rollout_workers=2, batch_mode="complete_episodes")
    .training(
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
        }
    )
)

algo = config.build()

stop = {
    "training_iteration": 10,
    "timesteps_total": 40000,
}

2024-05-19 19:24:33,745	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2024-05-19 19:24:36,105	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=13856) 2024-05-19 19:24:35,703	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=13856) 2024-05-19 19:24:35,703	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(PPO pid=13860) 2024-05-19 19:24:42,416	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=13860) 2024-05-19 19:24:42,604	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=13866) 2024-05-19 19:24:44,699	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to

In [8]:
os.chdir('/Users/umsaka/Documents/DDCPS/qdtree/src')

In [9]:
tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(
        local_dir="/Users/umsaka/Documents/DDCPS/qdtree/src/results",
        stop=stop,
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
    ),
)

result = tuner.fit()

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_QdTreeEnv_bad10_00000,40270,"{'ObsPreprocessorConnector_ms': 0.0023609958589076996, 'StateBufferConnector_ms': 0.002108979970216751, 'ViewRequirementAgentConnector_ms': 0.04327511414885521}","{'num_env_steps_sampled': 40270, 'num_env_steps_trained': 40270, 'num_agent_steps_sampled': 40270, 'num_agent_steps_trained': 40270}",{},2024-05-19_19-26-15,True,15.6641,{},0.48514,0.303888,0.129307,256,2398,dd58b9a45eae4e868e4fdda8b40f4a7f,Mete-New.local,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.860827862831854, 'cur_kl_coeff': 0.6750000000000002, 'cur_lr': 5.0000000000000016e-05, 'total_loss': -0.06883565503553117, 'policy_loss': -0.09064365520983213, 'vf_loss': 0.00855401635360253, 'vf_explained_var': 0.033846884132713397, 'kl': 0.01963553222075952, 'entropy': 4.348961020541448, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 8835.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 40270, 'num_env_steps_trained': 40270, 'num_agent_steps_sampled': 40270, 'num_agent_steps_trained': 40270}",10,127.0.0.1,40270,40270,40270,4010,40270,4010,0,2,0,0,4010,"{'cpu_util_percent': 21.40833333333333, 'ram_util_percent': 63.116666666666674}",13860,{},{},{},"{'mean_raw_obs_processing_ms': 0.15513650456068706, 'mean_inference_ms': 0.41910595969507736, 'mean_action_processing_ms': 0.04144881000508645, 'mean_env_wait_ms': 0.5871797523901627, 'mean_env_render_ms': 0.0}",90.2455,8.85839,90.2455,"{'training_iteration_time_ms': 9020.378, 'load_time_ms': 1.276, 'load_throughput': 3156977.722, 'learn_time_ms': 6542.961, 'learn_throughput': 615.471, 'synch_weights_time_ms': 2.453}",1716168375,0,40270,10,bad10_00000,2.46925


2024-05-19 19:26:15,822	INFO tune.py:798 -- Total run time: 95.44 seconds (95.23 seconds for the tuning loop).


In [10]:
agent = config.build()
agent.restore(result.get_best_result().checkpoint) # type: ignore

env = QdTreeEnv(EnvContext(env_config, 0))
done = False
obs, _ = env.reset() 
step = 0
episode_reward = 0
while not done:
    action = agent.compute_single_action(obs, explore=False)
    obs, reward, done, _, info = env.step(action) # type: ignore
    episode_reward += reward
    # print(step, action, obs, reward, done, info)
    step += 1

pprint(env.qd_tree.blocks)
print(episode_reward)


2024-05-19 19:30:03,665	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2024-05-19 19:30:03,672	WARNING algorithm_config.py:596 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-05-19 19:30:03,694	INFO trainable.py:791 -- Restored on 127.0.0.1 from checkpoint: /Users/umsaka/Documents/DDCPS/qdtree/src/results/PPO/PPO_QdTreeEnv_bad10_00000_0_2024-05-19_19-24-40/checkpoint_000010
2024-05-19 19:30:03,694	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 10, '_timesteps_total': None, '_time_total': 90.24550175666809, '_episodes_total': 2398}


[{'c_acctbal': (-inf, inf),
  'c_custkey': (-inf, inf),
  'c_nationkey': (-inf, inf),
  'l_commitdate': (-inf, inf),
  'l_discount': (-inf, inf),
  'l_extendedprice': (-inf, inf),
  'l_linenumber': (-inf, inf),
  'l_orderkey': (-inf, inf),
  'l_partkey': (-inf, inf),
  'l_quantity': (-inf, inf),
  'l_receiptdate': (-inf, inf),
  'l_shipdate': [788943600, 807256800),
  'l_suppkey': (-inf, inf),
  'l_tax': (-inf, inf),
  'n_nationkey_cust': (-inf, inf),
  'n_nationkey_supp': (-inf, inf),
  'n_regionkey_cust': (-inf, inf),
  'n_regionkey_supp': (-inf, inf),
  'o_custkey': (-inf, inf),
  'o_orderdate': [796460400, inf),
  'o_orderkey': (-inf, inf),
  'o_shippriority': (-inf, inf),
  'o_totalprice': (-inf, inf),
  'p_partkey': (-inf, inf),
  'p_retailprice': (-inf, inf),
  'p_size': (-inf, inf),
  'ps_availqty': (-inf, inf),
  'ps_partkey': (-inf, inf),
  'ps_suppkey': (-inf, inf),
  'ps_supplycost': (-inf, inf),
  'r_regionkey_cust': (-inf, inf),
  'r_regionkey_supp': (-inf, inf),
  's_acc

In [11]:
print(env.qd_tree)

{'id': 1,
 'cut': l_shipdate < 788943600,
 'size': 6001,
 'block': {'l_orderkey': (-inf, inf),
           'l_partkey': (-inf, inf),
           'l_suppkey': (-inf, inf),
           'l_linenumber': (-inf, inf),
           'l_quantity': (-inf, inf),
           'l_extendedprice': (-inf, inf),
           'l_discount': (-inf, inf),
           'l_tax': (-inf, inf),
           'l_shipdate': (-inf, inf),
           'l_commitdate': (-inf, inf),
           'l_receiptdate': (-inf, inf),
           'o_orderkey': (-inf, inf),
           'o_custkey': (-inf, inf),
           'o_totalprice': (-inf, inf),
           'o_orderdate': (-inf, inf),
           'o_shippriority': (-inf, inf),
           'c_custkey': (-inf, inf),
           'c_nationkey': (-inf, inf),
           'c_acctbal': (-inf, inf),
           'n_nationkey_cust': (-inf, inf),
           'n_regionkey_cust': (-inf, inf),
           'r_regionkey_cust': (-inf, inf),
           'ps_partkey': (-inf, inf),
           'ps_suppkey': (-inf, inf),
   